In [1]:
"""Importing relavant modules"""

%matplotlib inline
import numpy as np
import pandas as pd #for data structures
from pandas import DataFrame
import matplotlib.pyplot as plt
import pycountry as pc #this package stores country names and codes

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
"""Import data"""

data = pd.read_csv('Parchute science - search 1.csv')

In [2]:
def scraper(data):
    
    """this function scrapes the input data column for country names and adds them to a new list after appending
    them with a numerical tag that identifies countries that are pulled from the same text."""
    
    text = [] #initialise empty list to store converted text data in
    for i in range(len(data)):
        text.append(str(data[i])) #convert data to string to allow pycountry to scrape it and store in list
        df = [] # initialise empty list to store scraped country names in
        for j in range(len(text)):
            for country in pc.countries: #using pycountry to scrape country names
                if country.name in text[j]:
                    if len(str(j)) == 1: #these if statements add a 5 digit unique identifier to each country name
                        index = '0000' + str(j)
                    elif len(str(j)) == 2:
                             index = '000' + str(j)
                    elif len(str(j)) == 3:
                             index = '00' + str(j)
                    elif len(str(j)) == 4:
                             index = '0' + str(j)
                    else:
                        index = str(j)
                        
                    df.append(index + country.name)

In [3]:
def sort(data, original_array):
    
    """This function loops through the scraped data and deletes countries with non-unique 5-digit identifiers,
    leaving just one country per data point in the correspoding original array."""

    """data argument = data passed through scraper function."""
    
    prev_val = None #initialising initial value to compare first 5-digit value to.
    lst = [] #initialise empty list to store unique values in.
    
    for i in range(len(data)):
        if data[i][0:5] != prev_val:
            lst.append(data[i]) #add element from list of countries if the 5 digit identifier is not the same as prev
            prev_val = data[i][0:5] #set compare value to current 5 digit identifier
            
    indices = []
    for j in range(len(lst)): #this for loop stores the identifiers in lst as integers
        indices.append(int(lst[j][0:5]))
    
    #print(indices)
    
    #creating list of missing indices in indices to ascertain which texts were not successfully scraped for a country
    missing = [x for x in range(indices[0], indices[-1]+1) if x not in indices] 
    if indices[0] != 0: #insert an initial index if first identifier was not already 0
        missing.insert(0, 0)
        

    #print(missing)
    
        
    for j in range(len(missing)):
        lst.insert(missing[j], "Unknown") #adding "Unknown" to list of countries at the missing indexes from lst
        
    while len(lst) < len(original_array): #appending "Unknown" to end of lst until final index matches length of data
        lst.append("Unknown")

        
         
    return lst

In [4]:
def crosscheck(text1, text2):
    
    """This function iterates through the list of countries scraped from two texts and replaces "Unknown" in the first
    list with the entry from the second list, if this entry is not "Unknown". """
    
    """Arguments in this function are already scraped and sorted with the previous functions"""
    
    for i in range(len(text1)):
        if text1[i] == 'Unknown':
            text1[i] = text2[i]
    return text1

In [5]:
def remove_identity(data):
    
    """This function removes the 5 digit identifiers"""
    
    df = []
    for i in range(len(data)):
        if data[i] == 'Unknown':
            df.append(data[i])
        else:
            df.append(data[i][5:])
        
    return df

In [6]:
def final(data):
    
    """Combining previous functions for ease"""
    
    return remove_identity(sort(scraper(data), data))